<a href="https://www.kaggle.com/code/rahulkumarroy92/deepfake-forensics?scriptVersionId=204245797" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install tqdm tensorflow_docs mtcnn imutils imageio

  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.4 MB/s eta 0:00:00
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25837 sha256=667195364ce6bc6bd30450d093ee46d666d9ea2ffa045dd4edf840156e5a40db
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [2]:
# !pip install imageio

In [3]:
base_dir = "/kaggle/input/mini-face-forensics/FF++/"

In [4]:
import os
import pandas as pd
import cv2 as cv
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import imageio

In [5]:
label_types = os.listdir('/kaggle/input/mini-face-forensics/FF++')
print (label_types)  

['fake', 'real']


In [6]:
# Example data
video_paths = []
labels = []  


for sub_folder in os.listdir(base_dir):
    label = sub_folder
    
    sub_folder = os.path.join(base_dir, sub_folder)
    
    for vid in os.listdir(sub_folder):
        video_paths.append(os.path.join(sub_folder, vid))
        labels.append(label)

In [7]:
# Print the first 10 and last 10 elements of video_paths
print("First 10 video paths:", video_paths[:10])
print("Last 10 video paths:", video_paths[-10:])

# Print the first 10 and last 10 elements of labels
print("First 10 labels:", labels[:10])
print("Last 10 labels:", labels[-10:])

First 10 video paths: ['/kaggle/input/mini-face-forensics/FF++/fake/02_13__exit_phone_room__CP5HFV3K.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/03_14__talking_against_wall__ZC2KYASW.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/03_15__outside_talking_pan_laughing__Y11NT1YX.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/07_26__walking_down_street_outside_angry__FGNGC2GT.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/07_03__hugging_happy__7NGMD8FT.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/07_09__walk_down_hall_angry__N9CWME71.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/01_12__outside_talking_pan_laughing__TNI7KUZ6.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/07_03__podium_speech_happy__6PHZRQ4H.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/07_02__walking_down_street_outside_angry__O4SXNLRL.mp4', '/kaggle/input/mini-face-forensics/FF++/fake/08_05__walk_down_hall_angry__FBICSP2C.mp4']
Last 10 video paths: ['/kaggle/input/mini-face-forensics/F

In [8]:
# Preparing the data
df = pd.DataFrame({'video_path': video_paths, 'label': labels})

In [9]:
print(df.head())
print(df.tail())

                                          video_path label
0  /kaggle/input/mini-face-forensics/FF++/fake/02...  fake
1  /kaggle/input/mini-face-forensics/FF++/fake/03...  fake
2  /kaggle/input/mini-face-forensics/FF++/fake/03...  fake
3  /kaggle/input/mini-face-forensics/FF++/fake/07...  fake
4  /kaggle/input/mini-face-forensics/FF++/fake/07...  fake
                                            video_path label
395  /kaggle/input/mini-face-forensics/FF++/real/13...  real
396  /kaggle/input/mini-face-forensics/FF++/real/06...  real
397  /kaggle/input/mini-face-forensics/FF++/real/02...  real
398  /kaggle/input/mini-face-forensics/FF++/real/04...  real
399  /kaggle/input/mini-face-forensics/FF++/real/12...  real


In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [11]:
print(train_df.head())
print(train_df.tail())

                                            video_path label
3    /kaggle/input/mini-face-forensics/FF++/fake/07...  fake
18   /kaggle/input/mini-face-forensics/FF++/fake/02...  fake
202  /kaggle/input/mini-face-forensics/FF++/real/08...  real
250  /kaggle/input/mini-face-forensics/FF++/real/14...  real
274  /kaggle/input/mini-face-forensics/FF++/real/13...  real
                                            video_path label
71   /kaggle/input/mini-face-forensics/FF++/fake/03...  fake
106  /kaggle/input/mini-face-forensics/FF++/fake/01...  fake
270  /kaggle/input/mini-face-forensics/FF++/real/11...  real
348  /kaggle/input/mini-face-forensics/FF++/real/11...  real
102  /kaggle/input/mini-face-forensics/FF++/fake/02...  fake


In [12]:
print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 320
Total videos for testing: 80


,video_path,label
162,/kaggle/input/mini-face-forensics/FF++/fake/09...,fake
63,/kaggle/input/mini-face-forensics/FF++/fake/06...,fake
251,/kaggle/input/mini-face-forensics/FF++/real/13...,real
310,/kaggle/input/mini-face-forensics/FF++/real/07...,real
230,/kaggle/input/mini-face-forensics/FF++/real/07...,real
349,/kaggle/input/mini-face-forensics/FF++/real/10...,real
356,/kaggle/input/mini-face-forensics/FF++/real/15...,real
253,/kaggle/input/mini-face-forensics/FF++/real/13...,real
346,/kaggle/input/mini-face-forensics/FF++/real/12...,real
323,/kaggle/input/mini-face-forensics/FF++/real/07...,real


## capturing the frames

In [13]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [14]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
#   except RuntimeError as e:
#     print(e)

## Feature extraction

In [15]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["label"]))
print(label_processor.get_vocabulary())

labels = train_df["label"].values
labels = label_processor(labels[..., None]).numpy()
labels

['fake', 'real']


array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [17]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [18]:
def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df["video_path"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["label"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(path)
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━

In [19]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "/kaggle/working/tmp/video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()


Epoch 1/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5058 - loss: 0.8151
Epoch 1: val_loss improved from inf to 0.67946, saving model to /kaggle/working/tmp/video_classifier.weights.h5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - accuracy: 0.5018 - loss: 0.8113 - val_accuracy: 0.5312 - val_loss: 0.6795
Epoch 2/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4167 - loss: 0.7436
Epoch 2: val_loss did not improve from 0.67946
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4221 - loss: 0.7412 - val_accuracy: 0.5521 - val_loss: 0.6878
Epoch 3/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5214 - loss: 0.7136
Epoch 3: val_loss did not improve from 0.67946
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5193 - loss: 0.7135 - val_accuracy: 0.5521 - val_loss: 0.6823
Epoch 4/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4612 - loss: 0.7327
Epoch 4: val_loss did not improve from 0.67946
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4649 - loss: 0.7311 

## Testing

In [20]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_path"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: /kaggle/input/mini-face-forensics/FF++/real/12__walking_down_indoor_hall_disgust.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
  real: 52.82%
  fake: 47.18%


In [21]:
# from IPython.display import HTML

# HTML("""
#     <video alt="test" width="520" height="440" controls>
#         <source src="${test_video}" type="video/mp4" style="height:300px;width:300px">
#     </video>
# """)

## Extract frames from videos

In [22]:
# # enhancing the image
# def unsharp_mask(image, sigma=1.0, strength=1.5):
#     blurred = cv.GaussianBlur(image, (0, 0), sigma)
#     sharpened = cv.addWeighted(image, 1 + strength, blurred, -strength, 0)
#     return sharpened

# def enhance_frame_with_unsharp_mask(frame):
#     return unsharp_mask(frame)

In [23]:
# import cv2 as cv
# import numpy as np

# # padding included if frames are less than max_frames


# def extract_frames(video_path, max_frames=20, target_size=(224, 224), frame_skip=5):
#     cap = cv.VideoCapture(video_path)
#     frames = []
#     frame_count = 0
#     frame_position = 0

#     while cap.isOpened() and frame_count < max_frames:
#         # Set the position to skip frames
#         cap.set(cv.CAP_PROP_POS_FRAMES, frame_position)
        
#         # Read the frame
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Detect faces
#         detections = detector.detect_faces(frame)
        
#         if detections:
#             # Get the bounding box of the first detected face
#             x, y, width, height = detections[0]['box']
#             face_region = frame[y:y+height, x:x+width]
            
#             enhanced_face = unsharp_mask(face_region)

#             resized_face = cv.resize(enhanced_face, target_size)
            
#             frames.append(resized_face)
#             frame_count += 1 

#         # Update frame position by the frame_skip value
#         frame_position += frame_skip

#     # Pad with the last frame if we have fewer frames than max_frames
#     if len(frames) < max_frames and frames:
#         last_frame = frames[-1]
#         frames.extend([last_frame] * (max_frames - len(frames)))

#     cap.release()
#     return np.array(frames)

In [24]:
# demo_frames_fake = extract_frames('/kaggle/input/mini-face-forensics/FF++/fake/01_11__meeting_serious__9OM3VE0Y.mp4')
# demo_frames_real = extract_frames('/kaggle/input/mini-face-forensics/FF++/real/01__meeting_serious.mp4')

In [25]:
# print(demo_frames_fake)

In [26]:
# import matplotlib.pyplot as plt

# def plot_frames(demo_frames):
#     num_frames = len(demo_frames)
#     cols = 5
#     rows = (num_frames + cols - 1) // cols  # Calculate the number of rows needed


#     fig, axes = plt.subplots(rows, cols, figsize=(15, 10))

#     for i in range(num_frames):
#         # Convert the normalized frame to 0-255 range for display
#         frame = demo_frames[i]

#         # Plot the frame in its subplot
#         ax = axes[i // cols, i % cols]
#         ax.imshow(frame)
#         ax.axis('off')

#     # Hide any remaining empty subplots
#     for j in range(num_frames, rows * cols):
#         fig.delaxes(axes[j // cols, j % cols])

#     plt.tight_layout()
#     plt.show()

In [27]:
# plot_frames(demo_frames_fake)

In [28]:
# plot_frames(demo_frames_real)

## Feature Extraction Using Pretrained CNN (VGG16)

In [29]:
# vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# def extract_features(frames):
#     frame_features = []
#     for frame in frames:
#         frame = np.expand_dims(frame, axis=0)  # add batch dimension
#         feature = vgg_model.predict(frame)
#         frame_features.append(feature)
#     return np.array(frame_features).reshape(len(frames), -1)  # flatten features

In [30]:
# demo_frames_real_features = extract_features(demo_frames_real)

# print(demo_frames_real_features.shape)

## Prepare the dataset

In [31]:
# from tqdm import tqdm

# def prepare_data(video_paths, labels, max_frames=20):
#     X, y = [], []
#     for video_path, label in tqdm(zip(video_paths, labels), total=len(video_paths), desc="Preparing Data"):
#         frames = extract_frames(video_path, max_frames=max_frames)
#         features = extract_features(frames)
#         X.append(features)
#         y.append(label)
#     return np.array(X), np.array(y)

In [32]:
# Prepare the data
# X, y = prepare_data(video_paths, labels)

In [33]:
# from sklearn.model_selection import train_test_split

# # Split the data into 80% train and 20% test
# X_train, X_test, y_train_labels, y_test_labels = train_test_split(
#     X, y, test_size=0.2, random_state=42  # test_size is 20% of the data
# )

## LSTM Model

In [34]:
# def build_lstm_model(input_shape, num_classes=1):
#     model = Sequential()
#     model.add(LSTM(256, return_sequences=True, input_shape=input_shape))
#     model.add(LSTM(256))
#     model.add(Dropout(0.1))
#     model.add(Dense(128, activation='relu'))
#     model.add(Dense(num_classes, activation='sigmoid'))
#     return model

# # Compile the model
# sequence_length = 20
# feature_vector_size = 7 * 7 * 512
# model = build_lstm_model((sequence_length, feature_vector_size))

In [35]:
# model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

In [36]:
# model.summary()

## Train the model

In [37]:
# history = model.fit(X_train, y_train_labels, 
#           batch_size=32, 
#           epochs=20, 
#           validation_data=(X_test, y_test_labels),
#           callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')])

In [38]:
# import matplotlib.pyplot as plt

# # Retrieve the history data for accuracy and loss
# history_dict = history.history
# train_loss = history_dict['loss']
# val_loss = history_dict['val_loss']
# train_accuracy = history_dict['accuracy']
# val_accuracy = history_dict['val_accuracy']
# epochs = range(1, len(train_loss) + 1)

# # Plot Training and Validation Loss
# plt.figure(figsize=(14, 5))

# plt.subplot(1, 2, 1)
# plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
# plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# # Plot Training and Validation Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(epochs, train_accuracy, 'bo-', label='Training Accuracy')
# plt.plot(epochs, val_accuracy, 'ro-', label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()